<a href="https://colab.research.google.com/github/Nik8x/Org_Chart_Data_Extract/blob/master/Org_Chart_Data_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import re

# !pip install minecart
import minecart

import json

In [0]:
doc = minecart.Document(open("org_chart.pdf", "rb"))

In [0]:
def convert(i):
  page = doc.get_page(i)
  content = []
  cord = []
  for i in range(0, len(page.letterings)): 
    content.append(str(page.letterings[i])) # words and their numeric locations as a rectangular axis
    cord.append(page.letterings[i].bbox[0]) # extracting only x axis distance from vertical axis
    
  listy = []
  for i in zip(cord, content):
    if i[0] < 500 and i[0] > 0:
      listy.append(i)
    else:
      pass
   
  return pd.DataFrame(listy, columns = ["xaxis", "content"])

In [0]:
df = pd.DataFrame()
for i in range(0, 1):
  try:
    df = df.append(convert(i))
  except:
    print("error in page number ", i + 1)
df = df.reset_index()
df = df.drop("index", axis = 1)

In [46]:
df1 = df.copy()
df1.shape

(48, 2)

In [0]:
# df1 = df1[df1["xaxis"] ! = 92.663400][......][.....] # for excluding the xaxis which we dont want

In [48]:
df1.xaxis.value_counts()

73.353695     8
60.408926     4
102.839007    4
61.128081     4
31.642773     4
44.587541     4
115.783778    4
74.072853     4
169.931713    1
267.480302    1
121.103688    1
175.720758    1
80.232715     1
280.425074    1
92.663400     1
156.986942    1
214.914645    1
105.608169    1
67.287944     1
158.933009    1
Name: xaxis, dtype: int64

In [49]:
df2 = df1.copy()
df2["xaxis"] = df2["xaxis"].apply(lambda x : int(x)) # converting floats to integers
df2 = df2.reset_index().drop("index", axis = 1)
df2.head()

,xaxis,content
0,31,Mckenzie\tGarcia
1,31,Gregory\tPLC
2,31,728-623-3573x537
3,31,ewingjennifer@harris.info
4,60,Amy\tW


In [50]:
df2.xaxis.value_counts()

73     8
61     4
115    4
44     4
74     4
102    4
31     4
60     4
80     1
169    1
267    1
156    1
92     1
214    1
280    1
158    1
105    1
175    1
121    1
67     1
Name: xaxis, dtype: int64

In [80]:
df2.tail()

,xaxis,content
43,115,William\tAustin
44,115,Smith\tPLC
45,115,+1-833-502-9518
46,115,daviswilliam@johnson-simmons.com
47,280,\t


In [0]:
axis = []
content = []
try:
  
  for i in range(len(df)):
    
    if df2.xaxis[i] not in axis:
      axis.append(df2.xaxis[i])
      content.append(df2.content[i])
      
    elif df2.xaxis[i] in axis:
      
      if df2.xaxis[i] < df2.xaxis[i - 1]:
        axis.append(df2.xaxis[i])
        content.append(df2.content[i])
      
      elif df2.xaxis[i] > df2.xaxis[i - 1]:
        axis.append(df2.xaxis[i])
        content.append(df2.content[i])
      
      elif df2.xaxis[i] == df2.xaxis[i - 1]:
        content[-1] = content[-1] + ";" + df2.content[i]
      
      else:
        pass
    
    else:
      pass

except:
  pass

In [58]:
dff = pd.DataFrame({"xaxis" : axis, "content" : content})
dff.head()

,xaxis,content
0,31,Mckenzie\tGarcia;Gregory\tPLC;728-623-3573x537...
1,60,Amy\tW
2,92,olf
3,60,"Lopez,\tShaw\tand\tJacobson;+1-186-265-9693x90..."
4,61,Ian\tBowen\tMD;V


In [0]:
content_1 = []
len_of_str = []
for i in range(len(dff)):
  x = dff["content"][i]
  if len(x) > 30:
    y = (re.findall(r"""(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0, 15}net(?:)|
                    (?:[a-zA-Z'-]+[^a-zA-Z'-]+){0, 15}com(?:)|
                    (?:[a-zA-Z'-]+[^a-zA-Z'-]+){0, 15}\([0-9]{3}\)\s[0-9]{3}-[0-9]{4}(?:)""", x.replace(";", "xzx")))
    z= len(x.replace(";", "xzx")) - len("".join(y))
    content_1.append(y)
    len_of_str.append(z)
  else:
    content_1.append(x)
    len_of_str.append(0)

In [62]:
# storing content_1 into dff
dff["content_updated"] = content_1
dff["possible_error_in_length"] = len_of_str
dff.drop("content", axis = 1, inplace = True)

list_of_orgs = list(df2[df2.xaxis == 28].content)
# convert list of strings to list of list in every row of content
dff["content_updated_1"] = dff.content_updated.apply(lambda x : [i.split("xzx") for i in x if x not in list_of_orgs]) # to exclude the organizations name
keys = ["name", "position", "contact"]

# to remove empty strings "" from begining of list with list
def del1(x):
  c = []
  for j in x:
    if j[0] != "":
      c.append(j)
    elif j[0] == "":
      c.append(j[1:])
  return c
dff["content_updated_2"] = dff.content_updated_1.apply(lambda x : [dict(zip(keys, l)) for l in x])
dff.head()

,xaxis,content_updated,possible_error_in_length,content_updated_1,content_updated_2
0,31,[],76,[],[]
1,60,Amy\tW,0,"[[A], [m], [y], [\t], [W]]","[{'name': 'A'}, {'name': 'm'}, {'name': 'y'}, ..."
2,92,olf,0,"[[o], [l], [f]]","[{'name': 'o'}, {'name': 'l'}, {'name': 'f'}]"
3,60,[],78,[],[]
4,61,Ian\tBowen\tMD;V,0,"[[I], [a], [n], [\t], [B], [o], [w], [e], [n],...","[{'name': 'I'}, {'name': 'a'}, {'name': 'n'}, ..."


In [63]:
dff_1 = dff.copy()
dff_1["ID"] = [int('{}{}'.format(i, j)) for i, j in zip(dff_1.xaxis, dff_1.index)] # to create unique ids
dff_1.head()

,xaxis,content_updated,possible_error_in_length,content_updated_1,content_updated_2,ID
0,31,[],76,[],[],310
1,60,Amy\tW,0,"[[A], [m], [y], [\t], [W]]","[{'name': 'A'}, {'name': 'm'}, {'name': 'y'}, ...",601
2,92,olf,0,"[[o], [l], [f]]","[{'name': 'o'}, {'name': 'l'}, {'name': 'f'}]",922
3,60,[],78,[],[],603
4,61,Ian\tBowen\tMD;V,0,"[[I], [a], [n], [\t], [B], [o], [w], [e], [n],...","[{'name': 'I'}, {'name': 'a'}, {'name': 'n'}, ...",614


In [0]:
# to input the names of organiztions under Cigna to empty places
for i in list_of_orgs:
  dff_1.loc[dff_1["content_updated"] == i, "content_updated_2"] = i
  dff_1.loc[dff_1["content_updated"] == i, "content_updated_1"] = i
  # dff_1["content_updated_2"][dff_1.content_updated == i] = i

In [65]:
dff_1.head()

,xaxis,content_updated,possible_error_in_length,content_updated_1,content_updated_2,ID
0,31,[],76,[],[],310
1,60,Amy\tW,0,"[[A], [m], [y], [\t], [W]]","[{'name': 'A'}, {'name': 'm'}, {'name': 'y'}, ...",601
2,92,olf,0,"[[o], [l], [f]]","[{'name': 'o'}, {'name': 'l'}, {'name': 'f'}]",922
3,60,[],78,[],[],603
4,61,Ian\tBowen\tMD;V,0,"[[I], [a], [n], [\t], [B], [o], [w], [e], [n],...","[{'name': 'I'}, {'name': 'a'}, {'name': 'n'}, ...",614


In [0]:
# creating parent_ID for each row by comparing current ID(xaxis) with previous smaller ID(xaxis)
listy = []
for i in range(len(dff_1)):
  try: 
    listy.append(list(dff_1[0:i].loc[dff_1.xaxis[i]].ID)[-1])
  except:
    listy.append(1111)
    
dff_1["parent_ID"] = listy

# len(dff_1.content_updated_2[7])
# dff_1["Number_of_Boxes"] = dff_1.content_updated_2.apply(lambda x : len(x) if type(x) is str)
Number_of_Boxes = []
for i in dff_1.content_updated_2:
  if type(i) is not str:
    Number_of_Boxes.append(len(i))
  else:
    Number_of_Boxes.append(0)
    
dff_1["Number_of_Boxes"] = Number_of_Boxes

In [0]:
dff_1[["content_updated_1", "ID", "parent_ID", "Number_of_Boxes", "possible_error_in_length"]].to_csv("org_chart.csv")

In [69]:
dff_1[["content_updated_1", "ID", "parent_ID", "Number_of_Boxes", "possible_error_in_length"]].head()

,content_updated_1,ID,parent_ID,Number_of_Boxes,possible_error_in_length
0,[],310,1111,0,76
1,"[[A], [m], [y], [\t], [W]]",601,1111,5,0
2,"[[o], [l], [f]]",922,1111,3,0
3,[],603,1111,0,78
4,"[[I], [a], [n], [\t], [B], [o], [w], [e], [n],...",614,1111,14,0


In [0]:
# using recursion to get a json for a parent_ID
def get_childrens(df, parent_id):
  data = df[df["parent_ID"] == parent_id].apply(lambda row : {
                                                                "ID" : row["ID"],
                                                                "content" : row["content_updated_1"],
                                                                "sub" : []
                                                             },
                                                                 axis = 1)
  data_list = list(data) if not data.empty else []
  
  for i, d in enumerate(data_list):
    d["sub"] = get_childrens(df, d["ID"])
  return data_list

In [0]:
x = json.dumps({
                "sub" : get_childrens(dff_1, 1111)
                }, indent = 2)
y = eval(x) # converting string dictionary to dictionary

with open("organiztion.json", "w", encoding = "utf-8") as f:
  json.dump(y, f, ensure_ascii = False, indent = 2)

In [73]:
dff_1.possible_error_in_length.value_counts()

0     18
79     2
78     2
76     2
40     2
33     1
Name: possible_error_in_length, dtype: int64

In [79]:
dff_1["content_updated_1"][dff_1["possible_error_in_length"] == 40][23]

[]